#### 尝试用gensim.doc2vec 建模、利用相似度做文本分类，但效果太差，f1只有35%左右，没有往后继续测试。

#### 1. 加载数据

In [3]:
with open("./Total_Train_Datat_cutword.txt", "r", encoding='Utf-8') as f:
    with open("./Total_Train_Datat_cutword_new_format.txt", "w", encoding='Utf-8') as ff:
        for line in f:
            if line:
                line = line.strip()
                ff.write(line.split("\t")[0] + "\t" + line.split("\t")[1].replace("龘", "").replace("/", " ") + "\n")

In [6]:
with open("./Total_Train_Datat_cutword_new_format.txt", "r", encoding='Utf-8') as ff:
    data = ff.readlines()
    print(len(data))

99890


#### 2. 文本向量化

In [7]:
import gensim 

In [12]:
def read_corpus(train_data):
    with open(train_data, "r", encoding='Utf-8') as ff:
        for i, line in enumerate(ff):
            yield(gensim.models.doc2vec.TaggedDocument(line.split("\t")[1].split(), [line.split("\t")[0] + '_%s' % i]))

In [13]:
train_corpus = list(read_corpus("./Total_Train_Datat_cutword_new_format.txt"))

In [110]:
print(train_corpus[0])

TaggedDocument(['用', '好', '定位', '腾讯', '阿里', '就是', '你', '的', '运营', '助手', '编者按', '本文', '来自', '微信', '公众', '号', '笔记', '侠', 'ID', 'Notesman', '分享', '人', '特劳特', '中国', '创始', '合伙人', '陈', '奇峰', '36', '氪', '授权', '转载', '2018', '年', '5', '月', '11', '日', '12', '日', '火种', '定位', '学会', '定位', '学习网', '主办', '511', '首届', '中国', '定位', '日', '大会', '上', '特劳特', '中国', '创始', '合伙人', '陈', '奇峰', '进行', '调动', '心智', '力量', '把握', '时代', '机遇', '主题', '分享', '笔记', '君邀', '先', '思考', '了解', '定位', '定位', '得到', '越来越', '关注', '运用', '定位', '进行', '创新', '创业', '今天', '大家', '报告', '一下', '特劳特', '中国', '公司', '中国', '定位', '实践', '观察', '发现', '主要', '讲', '三点', '中国', '定位', '发展', '推进', '阶段', '定位', '1969', '年', '诞生', '美国', '现已', '发展', '成', '一门', '社会学科', '研究', '消费者', '产品', '服务', '认知', '顾客', '认知', '接受', '角度', '指引', '企业', '去', '设计', '产品', '服务', '经营', '模式', '实现', '更好', '经营', '中国', '发展', '目前', '进入', '第三个', '阶段', '1', '定位', '引领', '企业', '做广告', '第一阶段', '20', '世纪', '90', '年代', '当时', '中国', '企业', '主要', '区域性', '供货', '范围', '市场', '普及', '范围', '基本', '局限', '百公里', '级', '社

In [26]:
model = gensim.models.doc2vec.Doc2Vec(train_corpus)

In [149]:
# 整个训练集生成了28万842个词构成的词典
print(len(model.wv.vocab))

280842


In [157]:
# ?model.build_vocab
?gensim.models.doc2vec.LabeledSentence

In [32]:
# model.build_vocab(train_corpus)

In [41]:
# model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [67]:
'''
    训练集的内容转成向量，对应标签转成整数0/1
'''
train_arrays = []
train_labels = []
for i in list(model.docvecs.doctags.keys()):
    if i.startswith("0"): 
        train_arrays.append(model.docvecs[i])
        train_labels.append(0)
    if i.startswith("1"): 
        train_arrays.append(model.docvecs[i])
        train_labels.append(1)

#### 从语料库中找最相似的一句话

In [93]:
sims = model.docvecs.most_similar([line_vec], topn=1)

/home/wujiaocan/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [94]:
print(sims)

[('1_13489', 0.2892393469810486)]


In [98]:
index = int(sims[0][0].split('_')[1])
print(train_corpus[index])

TaggedDocument(['李军', '综合', '创新', '带动', '企业', '高速', '增长', '李军', '综合', '创新', '带动', '企业', '高速', '增长', '简介', '九三', '阅兵', '天安门广场', '朱日', '训练', '基地', '从大屏', '电视机', '巨幅', '户外广告', '屏', '一家', '不太', '知名', '中关村', '企业', '利亚德', '集团', 'Ｌ', 'Ｅ', 'Ｄ', '显示屏', '竟', '悄然', '占据', '１', '４', '全球', '市场份额', '利亚德', '集团', '创始人', '董事长', '兼', '总裁', '李军', '接受', '新华社', '记者', '采访', '时说', '综合', '创新', '带动', '企业', '高速', '增长', '记者', '李德', '欣', '王君璐', '马晓冬', '编辑', '祁', '星', '魏', '寅', '新华社', '音视频', '部', '制作'], ['1_13489'])


#### 3. 测试百度数据

In [137]:
'''
    测试集的内容转成向量
'''
import jieba
test_arrays = []
test_labels = []
with open("Baidu_TestData_1_format.txt", "r", encoding="utf-8") as w:
    for line in w:
        label = int(line.strip().split("\t")[0])
        line = " ".join([word for word in jieba.cut(line.strip().split("\t")[1])])
        line_vec = model.infer_vector(line, alpha=0.95, steps=50)
        test_arrays.extend(line_vec.reshape(1, -1))
        test_labels.append(label)

In [102]:
from sklearn.svm import LinearSVC
svm = LinearSVC()
svm.fit(train_arrays, train_labels)

In [121]:
y_prediction = svm.predict(test_arrays)
print(y_prediction)

[0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 1 1 0 1 1 1 0 1 1
 0 0 1 0 0 0 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1
 0 1 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 1 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1
 0 1 0 0 1 1 0 1 1 0 1 0 0 1 1 1 1 0 0 1 0 0 0 0 1 1 1 0 0 0 1 0 1 1 0 0 1
 1 1 0 1 1 1 0 1 0 0 0 1 0 1 1 1 1 0 1 1 0 0 1 1 0 1 1 1 1 0 0 0 0 0 1 1 0
 0 1 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 0 1 0 0 0 1 1 0 1 1
 0 0 1 1 1 0 1 1 0 0 1 0 0 0 0 0 1 1 0 0 1 1 0 1 1 1 0 0 0 0 1 0 1 1 1 1 0
 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 0
 0 0 1 0 1 0 0 0 1 0 1 0 0 1 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 0 0 1 1 1
 0 0 0 0 1 0 1 1 0 1 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 0
 0 0 1 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 0 0 0 0 1
 1 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0 1 1 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 1 1 

In [138]:
from sklearn.metrics import classification_report
print(classification_report(y_true=test_labels, y_pred=y_prediction))

             precision    recall  f1-score   support

          0       0.38      0.46      0.42       250
          1       0.33      0.26      0.29       250

avg / total       0.36      0.36      0.36       500



In [124]:
y_prediction2 = svm.predict(test_arrays)
print(y_prediction2)

[0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 1 0 0 0 1 0 1 0 1 0 0 0 1 0 1 0 1 1 1 1 1 1
 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 0 0 1 0 1
 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1
 0 0 1 1 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 0 1
 1 1 0 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 0 0 0 1 0 1 1 0 0 1
 1 1 0 1 0 1 0 1 1 1 0 0 0 1 1 1 1 0 1 1 0 0 1 1 0 1 1 1 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 1 1
 0 0 1 1 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 1 1 0 0 0 0 1 0 1 1 1 1 0
 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 1 1 0 1 0 1 0 0 1 0 0 0 0 1 1 1
 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 1 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 1 0 0
 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 1 1 1 0 0 1 1 1 0 0 1 0 0 0 0 0 1
 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 0 0 1 0 0 1 1 1 0 0 0
 0 0 1 0 1 0 0 0 0 0 0 1 

In [139]:
from sklearn.metrics import classification_report
print(classification_report(y_true=test_labels, y_pred=y_prediction2))

             precision    recall  f1-score   support

          0       0.38      0.46      0.42       250
          1       0.32      0.25      0.28       250

avg / total       0.35      0.36      0.35       500



In [140]:
print(test_labels)

[1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 